### Расчет продуктовых метрик для дашборда

In [1]:
# Импорт  библиотек
import pandahouse

In [2]:
# подключимся к ClickHouse

connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20250820'
}

In [3]:
# посчитаем retention для рекламного трафика (ads) для когорты 15 августа 2025

q_ads = """
WITH cohort AS (
  SELECT
    user_id,
    min(toDate(time)) AS start_date
  FROM simulator_20250820.feed_actions
  WHERE source = 'ads'
  GROUP BY user_id
  HAVING start_date = '2025-08-15'
)

SELECT
  toDate(t2.time) AS day,
  DATEDIFF('day', start_date, day) as days_after,
  COUNT(DISTINCT t2.user_id) AS retained_users,
  ROUND((COUNT(DISTINCT t2.user_id) * 100.0 / (SELECT COUNT(user_id) FROM cohort)), 2) AS retention_rate
FROM cohort AS t1
JOIN simulator_20250820.feed_actions AS t2 ON t1.user_id = t2.user_id
WHERE t2.time >= t1.start_date
GROUP BY day, days_after
ORDER BY day, days_after

"""

df_ads = pandahouse.read_clickhouse(q_ads, connection=connection)
df_ads

,day,days_after,retained_users,retention_rate
0,2025-08-15,0,2592,100.00
1,2025-08-16,1,103,3.97
2,2025-08-17,2,105,4.05
3,2025-08-18,3,96,3.70
4,2025-08-19,4,90,3.47
5,2025-08-20,5,84,3.24
6,2025-08-21,6,83,3.20
7,2025-08-22,7,75,2.89
8,2025-08-23,8,69,2.66
9,2025-08-24,9,234,9.03


In [4]:
# посчитаем retention для рекламного трафика (organic) для когорты 15 августа 2025

q_organic = """
WITH cohort AS (
  SELECT
    user_id,
    min(toDate(time)) AS start_date
  FROM simulator_20250820.feed_actions
  WHERE source = 'organic'
  GROUP BY user_id
  HAVING start_date = '2025-08-15'
)

SELECT
  toDate(t2.time) AS day,
  DATEDIFF('day', start_date, day) as days_after,
  COUNT(DISTINCT t2.user_id) AS retained_users,
  ROUND((COUNT(DISTINCT t2.user_id) * 100.0 / (SELECT COUNT(user_id) FROM cohort)), 2) AS retention_rate
FROM cohort AS t1
JOIN simulator_20250820.feed_actions AS t2 ON t1.user_id = t2.user_id
WHERE t2.time >= t1.start_date
GROUP BY day, days_after
ORDER BY day, days_after

"""

df_organic = pandahouse.read_clickhouse(q_organic, connection=connection)
df_organic

,day,days_after,retained_users,retention_rate
0,2025-08-15,0,741,100.00
1,2025-08-16,1,222,29.96
2,2025-08-17,2,236,31.85
3,2025-08-18,3,239,32.25
4,2025-08-19,4,218,29.42
5,2025-08-20,5,207,27.94
6,2025-08-21,6,201,27.13
7,2025-08-22,7,195,26.32
8,2025-08-23,8,165,22.27
9,2025-08-24,9,166,22.40


Для когорты 15.08.2025 выяснено:  
- в первый день пользователей с рекламного трафика было больше, чем с органического;  
- в долгосрочной перспективе доля удержанных рекламных пользователей меньше, чем органических.

In [5]:
# посчитаем колебания активной аудитории по неделям, в разрезе: новые пользователи, вернувшиеся и ушедшие.

q_users = """
SELECT 
  this_week, 
  previous_week, 
  -uniq(user_id) as num_users, 
  status 
FROM
  (SELECT user_id,
          groupUniqArray(toMonday(toDate(time))) as weeks_visited,
          addWeeks (arrayJoin(weeks_visited), +1) this_week,
          IF (has (weeks_visited, this_week) =1, 'retained', 'gone') as status,
          addWeeks (this_week, -1) as previous_week
   FROM simulator_20250820.feed_actions
   GROUP BY user_id)
WHERE status = 'gone'
GROUP BY this_week, previous_week, status
HAVING this_week != addWeeks (toMonday (today()), +1)
UNION ALL
SELECT 
  this_week, 
  previous_week, 
  toInt64 (uniq(user_id)) as num_users, 
  status
FROM
  (SELECT user_id,
          groupUniqArray(toMonday(toDate(time))) as weeks_visited,
          arrayJoin(weeks_visited) this_week,
          IF (has (weeks_visited, addWeeks(this_week, -1)) = 1, 'retained','new') as status,
          addWeeks (this_week, -1) as previous_week
   FROM simulator_20250820.feed_actions
   GROUP BY user_id)
GROUP BY this_week, previous_week, status
"""

df_users = pandahouse.read_clickhouse(q_users, connection=connection)
df_users

,this_week,previous_week,num_users,status
0,2025-08-18,2025-08-11,-20493,gone
1,2025-08-04,2025-07-28,-12208,gone
2,2025-07-28,2025-07-21,-8357,gone
3,2025-07-07,2025-06-30,-25,gone
4,2025-08-11,2025-08-04,-15257,gone
5,2025-07-14,2025-07-07,-1230,gone
6,2025-07-21,2025-07-14,-4393,gone
7,2025-09-01,2025-08-25,-23973,gone
8,2025-08-25,2025-08-18,-20892,gone
9,2025-09-08,2025-09-01,-27811,gone
